Notebook highlighting the steps to convert a dictionary (typical output from climt) to an xarray dataset

In [1]:
# load modules and the climt dictionary
# Note - this only works on a kernel with climt installed. See the link below for a guide to get climt_env on jupyter
# https://www.geeksforgeeks.org/python/using-jupyter-notebook-in-virtual-environment/

import pickle
import gzip

with gzip.open('climt_dict', 'rb') as f:
    climt_dict=pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'climt_dict'

In [ ]:
climt_dict

In [ ]:
# function to convert state (dictionary) to xarray dataset

import xarray as xr
def format_data(state):
    arr=[]
    for i in state.keys():
        if i!='time':
            # list of fields you wish to include
            if i in ['air_temperature','air_pressure','specific_humidity',
            'northward_wind','eastward_wind','surface_air_pressure', 'surface_temperature',
            'surface_upward_latent_heat_flux', 'surface_upward_sensible_heat_flux',
            'boundary_layer_height', 'divergence_of_wind','upwelling_longwave_flux_in_air',
            'downwelling_longwave_flux_in_air']:

                # fields you want only one level of (the lowest level in this case)
                if i in ['upwelling_longwave_flux_in_air','downwelling_longwave_flux_in_air']:
                    arr.append(state[i][0].rename('surface_'+i).astype('float32'))
                else:
                    arr.append(state[i].rename(i).astype('float32'))

    data=xr.merge(arr)

    # sets lat-lon coordinates
    lats=state['latitude'].values[:,0]; lons=state['longitude'].values[0,:]
    data=data.assign_coords({"lat": lats, 'lon': lons})

    # creates new dimension with time
    data = data.expand_dims(time=[state['time']])

    return(data)

climt_data=format_data(climt_dict)

In [ ]:
# the data is now a xarray dataset, with all the benefits that come with it!

climt_data

In [ ]:
# data can now be plotted with the .plot() command

climt_data['boundary_layer_height'].plot()

In [ ]:
# Storing this array to file using zarr

import zarr

store = zarr.storage.DirectoryStore("climt_run/run1")
climt_data.to_zarr(store=store, mode='w')

In [ ]:
# loading from a zarr file

climt_data=xr.open_zarr("climt_run/run1",consolidated=False)
climt_data